Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [54]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [93]:
batch_size = 10
layer1_depth = 1024
lambda1 = 0.01
lambda2 = 0.0

graph = tf.Graph()
with graph.as_default():
    tf_train_data = tf.placeholder(shape=[batch_size,image_size*image_size],dtype=tf.float32)
    tf_train_labels = tf.placeholder(shape=[batch_size,num_labels],dtype=tf.float32)
    tf_validation_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,layer1_depth]))
    bias = tf.Variable(tf.zeros(layer1_depth))
    activation1 = tf.nn.relu(tf.matmul(tf_train_data,weights) + bias)
    weights2 = tf.Variable(tf.truncated_normal([layer1_depth,num_labels]))
    bias2 = tf.Variable(tf.zeros(num_labels))
    logits = tf.matmul(activation1,weights2)+bias2
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits) + 
        lambda1*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2))
    )
                       
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.relu(tf.matmul(tf_validation_data,weights)+bias+lambda1*tf.nn.l2_loss(weights)),weights2
            )+bias2+lambda2*tf.nn.l2_loss(weights2)
    )
    
    test_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_data,weights)+bias+lambda1*tf.nn.l2_loss(weights)),weights2
            )+bias2+lambda2*tf.nn.l2_loss(weights2)
        )
    

In [98]:
train_loop = 6001
train_dataset_size = train_dataset.shape[0]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    for step in range(train_loop):
        offset = (batch_size*step)%(train_dataset_size-batch_size)
        tf_train_batch_data = train_dataset[offset:offset+batch_size,:]
        tf_train_batch_labels = train_labels[offset:offset+batch_size,:]
#         print(tf_train_batch_labels.shape)
        
        feed_dict = {tf_train_data:tf_train_batch_data,tf_train_labels:tf_train_batch_labels}
        
        _,l,prediction = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        
        if step%500 == 0:
            print("Loss at step {} is {}:".format(step,l) )
#             print(prediction.shape, tf_train_batch_labels.shape)
            print("mini batch accuracy:{}".format(accuracy(prediction,tf_train_batch_labels)))
            print("validation accuracy:{}".format(accuracy(validation_prediction.eval(),valid_labels)))
    print("Test accuracy:{}".format(accuracy(test_prediction.eval(),test_labels)))

Loss at step 0 is 3479.981201171875:
mini batch accuracy:20.0
validation accuracy:10.0
Loss at step 500 is 187.0621337890625:
mini batch accuracy:30.0
validation accuracy:16.27
Loss at step 1000 is 2.2968125343322754:
mini batch accuracy:70.0
validation accuracy:65.39
Loss at step 1500 is 1.012149453163147:
mini batch accuracy:70.0
validation accuracy:66.02
Loss at step 2000 is 1.3889023065567017:
mini batch accuracy:50.0
validation accuracy:57.41
Loss at step 2500 is 1.5042996406555176:
mini batch accuracy:70.0
validation accuracy:58.88
Loss at step 3000 is 0.9365217089653015:
mini batch accuracy:80.0
validation accuracy:59.35
Loss at step 3500 is 1.3343483209609985:
mini batch accuracy:70.0
validation accuracy:65.6
Loss at step 4000 is 2.036003828048706:
mini batch accuracy:30.0
validation accuracy:59.83
Loss at step 4500 is 2.0993194580078125:
mini batch accuracy:40.0
validation accuracy:63.77
Loss at step 5000 is 1.7451015710830688:
mini batch accuracy:50.0
validation accuracy:45.7

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
batch_size = 10
layer1_depth = 1024
lambda1 = 0.01
lambda2 = 0.0

graph = tf.Graph()
with graph.as_default():
    tf_train_data = tf.placeholder(shape=[batch_size,image_size*image_size],dtype=tf.float32)
    tf_train_labels = tf.placeholder(shape=[batch_size,num_labels],dtype=tf.float32)
    tf_validation_data = tf.constant(valid_dataset)
    tf_test_data = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,layer1_depth]))
    bias = tf.Variable(tf.zeros(layer1_depth))
    activation1 = tf.nn.relu(tf.matmul(tf_train_data,weights) + bias)
    
    
    weights2 = tf.Variable(tf.truncated_normal([layer1_depth,num_labels]))
    bias2 = tf.Variable(tf.zeros(num_labels))
    logits = tf.matmul(activation1,weights2)+bias2
    
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits) + 
        lambda1*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(weights2))
    )
                       
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    validation_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.relu(tf.matmul(tf_validation_data,weights)+bias+lambda1*tf.nn.l2_loss(weights)),weights2
            )+bias2+lambda2*tf.nn.l2_loss(weights2)
    )
    
    test_prediction = tf.nn.softmax(
    tf.matmul(
            tf.nn.relu(tf.matmul(tf_test_data,weights)+bias+lambda1*tf.nn.l2_loss(weights)),weights2
            )+bias2+lambda2*tf.nn.l2_loss(weights2)
        )
    

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
